In [ ]:
import pandas as pd # manipular dados em formato de texto
#scikit-learn: utilitários de machine learning
from sklearn.model_selection import train_test_split #separar dados em treino e teste
from sklearn.feature_extraction.text import CountVectorizer #transformar texto em números (contagem de palavras)
from sklearn.ensemble import RandomForestClassifier # algoritimo de classificação random forest
from sklearn.preprocessing import LabelEncoder # converter textos em números
from sklearn.metrics import classification_report #gerar métricas de avaliação do modelo
#NLTK : processamento de linguagem natual
import nltk # baixar recursos
from nltk.corpus import stopwords #lista de stopwords (palavras comuns)
nltk.download('stopwords') # Baixar stopwords em português
stopwords_pt = stopwords.words('portuguese') # Aqui serve para ignorar as palavras comuns como: "de"; "o". "que" etc

# 1. Carregar os dados 
caminho_arquivo = "c:\\Users\\..."
df = pd.read_excel(caminho_arquivo)

# 2. Criar campo de texto combinado 
df["texto_completo"] = df["DNA do Texto Digitado"].astype(str) + " | " + df["Jornada Programada"].astype(str)

df["texto_completo"] = df.apply(
    lambda row: row["texto_completo"] + " | " + row["NE"]
    if pd.notnull(row["NE"]) and row["NE"].strip() != ""
    else row["texto_completo"],
    axis=1
)

# 3. Separar base com e sem rótulo === obs: certifique de mudar/ajustar o separador "-" de acordo com a planilha
df_treino = df[df["Jornada após NE (manual)"].notna() & (df["Jornada após NE (manual)"].str.strip() != "-")].copy()
df_prever = df[df["Jornada após NE (manual)"].isna() | (df["Jornada após NE (manual)"].str.strip() == "-")].copy()

# 4. Codificar a variável alvo
label_encoder = LabelEncoder()
df_treino["label"] = label_encoder.fit_transform(df_treino["Jornada após NE (manual)"])

# 5. Remover classes com menos de 10 ocorrências
classe_counts = df_treino["label"].value_counts()
classes_validas = classe_counts[classe_counts >= 10].index
df_treino = df_treino[df_treino["label"].isin(classes_validas)]

# 6. Vetorização com CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1, 3), stop_words=stopwords_pt)
X_treino = vectorizer.fit_transform(df_treino["texto_completo"])
y_treino = df_treino["label"]

# 7. Treinar modelo
clf = RandomForestClassifier(random_state=42, class_weight="balanced")
clf.fit(X_treino, y_treino)

#  8. Prever para as linhas sem rótulo
X_prever = vectorizer.transform(df_prever["texto_completo"])
y_pred = clf.predict(X_prever)
df_prever["Previsão_Jornada"] = label_encoder.inverse_transform(y_pred)

#  9. Juntar de volta os dados
df_saida = df.copy()
df_saida.loc[df_prever.index, "Previsão_Jornada"] = df_prever["Previsão_Jornada"]

# 10. Salvar novo Excel com as previsões
df_saida.to_excel("Base_Teste_Preenchida_com_Prev.xlsx", index=False)
print("Arquivo salvo como Base_Teste_Preenchida_com_Prev.xlsx")











# Para visuzlizar a arvore
#baixar 
pip install matplotlib
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
plot_tree(
    clf.estimators_[5],  # 5 árvores da floresta
    feature_names=vectorizer.get_feature_names_out(),
    class_names=label_encoder.classes_,
    filled=True,
    rounded=True,
    max_depth=5 # Limita a profundidade para visualização
)
plt.show()